In [1]:
import pandas as pd
import numpy as np

In [2]:
merged_df = pd.read_csv(r"C:\Users\mynan\Desktop\capstone project\airbnb\final_df.xls")
merged_df

,listing_id,latitude,longitude,property_type,room_type,accomodates,bedrooms,beds,host_id,bathroom_number,...,host_since,Host_Age,distance_to_airport,distance_to_railway_station,month,day_of_week,listing_month,listing_year,host_since_month,host_since_year
0,50904,51.218575,4.398631,Room in boutique hotel,Hotel room,2,1.0,1.0,234077,1.0,...,2010-09-14,165,5.383636,1.572374,6,3,6,2022,9,2010
1,50904,51.218575,4.398631,Room in boutique hotel,Hotel room,2,1.0,1.0,234077,1.0,...,2010-09-14,165,5.383636,1.572374,1,6,1,2022,9,2010
2,50904,51.218575,4.398631,Room in boutique hotel,Hotel room,2,1.0,1.0,234077,1.0,...,2010-09-14,165,5.383636,1.572374,5,1,5,2022,9,2010
3,50904,51.218575,4.398631,Room in boutique hotel,Hotel room,2,1.0,1.0,234077,1.0,...,2010-09-14,165,5.383636,1.572374,5,1,5,2022,9,2010
4,50904,51.218575,4.398631,Room in boutique hotel,Hotel room,2,1.0,1.0,234077,1.0,...,2010-09-14,165,5.383636,1.572374,10,3,10,2022,9,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318220,53983318,51.214580,4.399620,Entire rental unit,Entire home/apt,3,1.0,1.0,437309332,1.5,...,2021-12-24,30,5.070786,1.524207,9,4,9,2022,12,2021
318221,53983318,51.214580,4.399620,Entire rental unit,Entire home/apt,3,1.0,1.0,437309332,1.5,...,2021-12-24,30,5.070786,1.524207,8,5,8,2022,12,2021
318222,53983318,51.214580,4.399620,Entire rental unit,Entire home/apt,3,1.0,1.0,437309332,1.5,...,2021-12-24,30,5.070786,1.524207,3,1,3,2022,12,2021
318223,53983318,51.214580,4.399620,Entire rental unit,Entire home/apt,3,1.0,1.0,437309332,1.5,...,2021-12-24,30,5.070786,1.524207,7,5,7,2022,12,2021


In [3]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318225 entries, 0 to 318224
Data columns (total 38 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   listing_id                   318225 non-null  int64  
 1   latitude                     318225 non-null  float64
 2   longitude                    318225 non-null  float64
 3   property_type                318225 non-null  object 
 4   room_type                    318225 non-null  object 
 5   accomodates                  318225 non-null  int64  
 6   bedrooms                     318225 non-null  float64
 7   beds                         318225 non-null  float64
 8   host_id                      318225 non-null  int64  
 9   bathroom_number              318225 non-null  float64
 10  bathroom_type                318225 non-null  object 
 11  Wifi                         318225 non-null  int64  
 12  Essentials                   318225 non-null  int64  
 13 

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
# Convert 'date' and 'host_since' columns to datetime
merged_df['date'] = pd.to_datetime(merged_df['date'], errors='coerce')
merged_df['host_since'] = pd.to_datetime(merged_df['host_since'], errors='coerce')

# Extract useful features from datetime columns
merged_df['listing_month'] = merged_df['date'].dt.month
merged_df['listing_year'] = merged_df['date'].dt.year
merged_df['host_since_month'] = merged_df['host_since'].dt.month
merged_df['host_since_year'] = merged_df['host_since'].dt.year

# Drop the original 'date' and 'host_since' columns
merged_df = merged_df.drop(columns=['date', 'host_since'])

# Handle missing values (example: fill with median, or drop)
merged_df.fillna(merged_df.median(numeric_only=True), inplace=True)

categorical_columns = ['property_type', 'room_type', 'bathroom_type']
merged_df = pd.get_dummies(merged_df, columns=categorical_columns, drop_first=True)

# Select features and target variable
target = 'price'
features = merged_df.columns[merged_df.columns != target]

X = merged_df[features]
y = merged_df[target]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Output shapes of the splits to verify
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)


X_train shape: (222757, 78)
X_val shape: (47734, 78)
X_test shape: (47734, 78)
y_train shape: (222757,)
y_val shape: (47734,)
y_test shape: (47734,)


In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [8]:
# Train Random Forest Regressor model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Predict on validation set
y_val_pred = rf_model.predict(X_val)

# Evaluate model
rf_mae = mean_absolute_error(y_val, y_val_pred)
rf_mse = mean_squared_error(y_val, y_val_pred)
rf_r2 = r2_score(y_val, y_val_pred)

print("Random Forest Regressor Performance:")
print("MAE:", rf_mae)
print("MSE:", rf_mse)
print("R²:", rf_r2)

Random Forest Regressor Performance:
MAE: 1.2869455733858466
MSE: 58.06711265345456
R²: 0.998500236546789


In [9]:
# Predict on test set
y_test_pred = rf_model.predict(X_test)

# Evaluate model on test set
test_rf_mae = mean_absolute_error(y_test, y_test_pred)
test_rf_mse = mean_squared_error(y_test, y_test_pred)
test_rf_r2 = r2_score(y_test, y_test_pred)

print("Test Set Performance for Random Forest Regressor:")
print("MAE:", test_rf_mae)
print("MSE:", test_rf_mse)
print("R²:", test_rf_r2)


Test Set Performance for Random Forest Regressor:
MAE: 1.4016939707545986
MSE: 121.63470813256798
R²: 0.9956058407503108


In [10]:
import joblib

# Save the trained model to a file
joblib.dump(rf_model, 'random_forest_model.pkl')

# Save the scaler to a file
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [11]:
# Define a function to take input from the user
def get_user_input():
    user_data = {
        'latitude': float(input("Enter latitude: ")),
        'longitude': float(input("Enter longitude: ")),
        'property_type': input("Enter property type: "),
        'room_type': input("Enter room type: "),
        'accommodates': int(input("Enter number of accommodates: ")),
        'bedrooms': float(input("Enter number of bedrooms: ")),
        'beds': float(input("Enter number of beds: ")),
        'bathroom_number': float(input("Enter number of bathrooms: ")),
        'top_10_amenities_count': int(input("Enter count of top 10 amenities: ")),
        'minimum_nights': int(input("Enter minimum nights: ")),
        'maximum_nights': int(input("Enter maximum nights: ")),
        'bathroom_type': input("Enter bathroom type: ")
    }
    return user_data

# Function to preprocess the user input
def preprocess_user_input(user_data, scaler, feature_names):
    # Convert user data to DataFrame
    user_df = pd.DataFrame([user_data])
    
    # Handle categorical variables
    categorical_columns = ['property_type', 'room_type', 'bathroom_type']
    user_df_encoded = pd.get_dummies(user_df, columns=categorical_columns, drop_first=True)
    
    # Align columns of user_df_encoded with the training data columns
    for col in feature_names:
        if col not in user_df_encoded.columns:
            user_df_encoded[col] = 0

    user_df_encoded = user_df_encoded[feature_names]
    
    # Scale the user input data
    user_df_scaled = scaler.transform(user_df_encoded)
    
    return user_df_scaled

# Predict price based on user input
def predict_price(user_data, model, scaler, feature_names):
    user_df_scaled = preprocess_user_input(user_data, scaler, feature_names)
    
    # Predict the price
    price_prediction = model.predict(user_df_scaled)
    return price_prediction[0]

# Get user input
user_input = get_user_input()

# Predict the price
predicted_price = predict_price(user_input, rf_model, scaler, features)

print(f"Predicted price for the given inputs: ${predicted_price:.2f}")


Enter latitude: 51.218575	
Enter longitude: 4.398631	
Enter property type: Room in boutique hotel	
Enter room type: Hotel room
Enter number of accommodates: 2
Enter number of bedrooms: 1.0
Enter number of beds: 1.0
Enter number of bathrooms: 1.0
Enter count of top 10 amenities: 9
Enter minimum nights: 1
Enter maximum nights: 60
Enter bathroom type: private bath
Predicted price for the given inputs: $94.09
